In [9]:
import requests

def get_company_data(org_number):
    url = f"https://data.brreg.no/regnskapsregisteret/regnskap/{org_number}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

org_number = "911382008"
company_data = get_company_data(org_number)

if company_data:
    print(company_data)
else:
    print("Failed to fetch company data.")
    
# save the data to a file

import json

with open('output/company_data.json', 'w') as outfile:
    json.dump(company_data, outfile, indent=4)


# convert the data to a dataframe 

    

[{'id': 3212571, 'journalnr': '2022419739', 'regnskapstype': 'SELSKAP', 'virksomhet': {'organisasjonsnummer': '911382008', 'organisasjonsform': 'ASA', 'morselskap': True}, 'regnskapsperiode': {'fraDato': '2021-01-01', 'tilDato': '2021-12-31'}, 'valuta': 'NOK', 'avviklingsregnskap': False, 'oppstillingsplan': 'store', 'revisjon': {'ikkeRevidertAarsregnskap': False, 'fravalgRevisjon': False}, 'regnkapsprinsipper': {'smaaForetak': False, 'regnskapsregler': 'regnskapslovenAlminneligRegler'}, 'egenkapitalGjeld': {'sumEgenkapitalGjeld': 27880000000.0, 'egenkapital': {'sumEgenkapital': 11283000000.0, 'opptjentEgenkapital': {'sumOpptjentEgenkapital': 5105000000.0}, 'innskuttEgenkapital': {'sumInnskuttEgenkaptial': 6178000000.0}}, 'gjeldOversikt': {'sumGjeld': 16597000000.0, 'kortsiktigGjeld': {'sumKortsiktigGjeld': 8787000000.0}, 'langsiktigGjeld': {'sumLangsiktigGjeld': 7810000000.0}}}, 'eiendeler': {'sumEiendeler': 27880000000.0, 'omloepsmidler': {'sumOmloepsmidler': 9095000000.0}, 'anleggsm

In [10]:
import json
import pandas as pd

def extract_numerical_values(data, prefix=''):
    numerical_data = []
    
    for key, value in data.items():
        if isinstance(value, dict):
            numerical_data.extend(extract_numerical_values(value, prefix + key + '.'))
        elif isinstance(value, (int, float)) and not isinstance(value, bool):
            
            value = int(value)
            numerical_data.append((key, value))
            
    return numerical_data

def json_to_dataframe(json_data):
    flattened_data = [item for record in json_data for item in extract_numerical_values(record)]
    
    df = pd.DataFrame(flattened_data, columns=['Variable', 'Value'])
    
    return df

df = json_to_dataframe(company_data)

df

,Variable,Value
0,id,3212571
1,sumEgenkapitalGjeld,27880000000
2,sumEgenkapital,11283000000
3,sumOpptjentEgenkapital,5105000000
4,sumInnskuttEgenkaptial,6178000000
5,sumGjeld,16597000000
6,sumKortsiktigGjeld,8787000000
7,sumLangsiktigGjeld,7810000000
8,sumEiendeler,27880000000
9,sumOmloepsmidler,9095000000


In [11]:
# save the dataframe to a excel file

df.to_excel('output/company_data.xlsx', index=False)

In [20]:
import PyPDF2

def read_pdf_file(file_path, start_page=1, end_page=100):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        total_pages = len(pdf_reader.pages)
        pdf_text = ''

        for page_num in range(start_page-1, end_page):
            pdf_page = pdf_reader.pages[page_num]
            pdf_text += pdf_page.extract_text()

    return pdf_text

file_path = 'elkem_data/2021.pdf'
start_page=6
end_page=9
pdf_data = read_pdf_file(file_path,
                         start_page=start_page,
                         end_page=end_page
                         )

print(f"Extracted data from pages {start_page}, {end_page} of the PDF:")
print(pdf_data)

# save the data to a file

with open('elkem_data/pdf_data.txt', 'w') as outfile:
    outfile.write(pdf_data)


incorrect startxref pointer(1)


Extracted data from pages 6, 9 of the PDF:
 
 
 
 
Konsernets resultatregnskap
 
 
 
Beløp i: NOK
Note 
2021 
2020 
 
 
 
RESULTATREGNSKAP
 
 
 
 
 
 
Inntekter
 
 
 
Revenue
 
33 083 000 000 
24 025 000 000 
Other operating income
 
586 000 000 
631 000 000 
Share of profit (loss) from equity accounted companies
 
48 000 000 
35 000 000 
Sum inntekter
 
33 717 000 000 
24 691 000 000 
 
 
 
Kostnader
 
 
 
Raw materials and energy for production
 
15 861 000 000 
12 858 000 000 
Employee benefit expenses
 
4 530 000 000 
4 028 000 000 
Amortisation and depreciation
 
1 816 000 000 
1 710 000 000 
Impairment losses
 
76 000 000 
17 000 000 
Other operating expenses
 
5 536 000 000 
5 121 000 000 
Other items
 
113 000 000 
130 000 000 
Sum kostnader
 
27 932 000 000 
23 864 000 000 
 
 
 
Driftsresultat
 
5 785 000 000 
827 000 000 
 
 
 
Finansinntekter og finanskostnader
 
 
 
Share of profit (loss) from equity accounted financial investments
 
37 000 000 
-15 000 000 
Finance income

Revenue: []
Other operating income: []
